<a href="https://colab.research.google.com/github/karthikmohan1702/EIP_4/blob/master/v4___PersonAttrubutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/EIP_4_Computer_vision/Week_5/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [2]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils




Using TensorFlow backend.


In [0]:
import numpy as np


def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
#def get_random_eraser(p=0.1, s_l=0.01, s_h=0.1, r_1=0.1, r_2=1/0.1, v_l=0, v_h=20, pixel_level=False):  
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [5]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [6]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(image,shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [8]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.25)
train_df.shape, val_df.shape

((10179, 28), (3394, 28))

In [9]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
7492,resized/7493.jpg,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0
9180,resized/9181.jpg,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
9299,resized/9300.jpg,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0
2554,resized/2555.jpg,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0
3310,resized/3311.jpg,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32,augmentation = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255,preprocessing_function=get_random_eraser()) )
valid_gen = PersonDataGenerator(val_df, batch_size=32,augmentation = ImageDataGenerator(rescale=1./255))

In [11]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [12]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D, Activation, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten, Dense
from keras.layers import Activation
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import AveragePooling2D, Input, Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np


inp = Input(shape = (224, 224, 3))
x = Convolution2D(32, (3, 3), padding = 'valid')(inp) #222
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(64, (3, 3))(x) #220
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x) #110
x = Dropout(0.2)(x)

x = Conv2D(128, (3, 3), padding = 'valid')(x) #108
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(64, (1, 1), padding = 'valid')(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x)#54
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3))(x)#52
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x)#26
x = Dropout(0.2)(x)

x = Conv2D(256, (3, 3))(x)#24
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x)#12
x = Dropout(0.2)(x)

x = Conv2D(512, (3, 3))(x)#10
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(128, (1, 1), padding = 'valid')(x)
x = Activation('relu')(x)

#x = MaxPooling2D(pool_size = (2, 2))(x)#8
#x = Dropout(0.2)(x)

#x = keras.layers.GlobalMaxPooling2D()(x)
x = AveragePooling2D(pool_size=8)(x)



x = Flatten(name="flatten")(x)
gender = Dense(num_units["gender"], activation="sigmoid", name=f"{'gender'}_output")(x)
image_quality = Dense(num_units["image_quality"], activation="softmax", name=f"{'image_quality'}_output")(x)
age = Dense(num_units["age"], activation="softmax", name=f"{'age'}_output")(x)
weight = Dense(num_units["weight"], activation="softmax", name=f"{'weight'}_output")(x)
bag = Dense(num_units["bag"], activation="softmax", name=f"{'bag'}_output")(x)
footwear = Dense(num_units["footwear"], activation="softmax", name=f"{'footwear'}_output")(x)
emotion = Dense(num_units["emotion"], activation="softmax", name=f"{'emotion'}_output")(x)
pose = Dense(num_units["pose"], activation="softmax", name=f"{'pose'}_output")(x)


# heads
"""gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))"""


model = Model(inputs=inp, outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion])

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 70:
        lr *= 0.5e-3
    elif epoch > 60:
        lr *= 1e-3
    elif epoch > 50:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1

    
    """lr = 1e-3
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 70:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 50:
        lr *= 1e-1"""



    """lr = 1e-3
    if epoch > 10:
        lr *= 1e-1
    elif epoch > 25:
        lr *= 1e-2
    elif epoch > 35:
        lr *= 1e-1"""

    
    """lr = 3e-3
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 70:
        lr *= 1e-3
    elif epoch > 45:
        lr *= 1e-2
    elif epoch > 35:
        lr *= 1e-1"""


    print('Learning rate: ', lr)
    return lr













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [13]:
import os
os.getcwd()

'/content'

In [14]:
from keras.callbacks import ModelCheckpoint
save_dir = os.path.join(os.getcwd(), 'gdrive/My Drive/EIP_4_Computer_vision/Week_5/saved_models')
print(save_dir)
model_name = 'v2_person_attr%s_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

/content/gdrive/My Drive/EIP_4_Computer_vision/Week_5/saved_models


In [0]:
callbacks = [checkpoint]

In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 222, 222, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 222, 222, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 222, 222, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
#from keras.utils import plot_model
#plot_model(model)

In [19]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
#opt = SGD(lr=0.001, momentum=0.9)
#model.compile(
 #   optimizer=opt,
  #  loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
   # metrics=["accuracy"]
#)

##########################
from keras.optimizers import Adam
losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "categorical_crossentropy",
 	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy",
    }
"""loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0,"weight_output":1.0,
                "bag_output":1.0,"pose_output":1.0,"footwear_output":1.0,"emotion_output":1.0 }"""
opt = SGD(lr=0.001, momentum=0.9)
#Adam(lr=lr_schedule(0))
model.compile(optimizer=Adam(lr=lr_schedule(0)),loss=losses, metrics=["accuracy"])

Learning rate:  0.001


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=2, 
    epochs=70,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/70
318/318 [==============================] - 134s 421ms/step - loss: 7.8196 - gender_output_loss: 0.6811 - image_quality_output_loss: 0.9477 - age_output_loss: 1.4390 - weight_output_loss: 0.9953 - bag_output_loss: 0.9274 - footwear_output_loss: 0.9805 - pose_output_loss: 0.9302 - emotion_output_loss: 0.9184 - gender_output_acc: 0.5697 - image_quality_output_acc: 0.5481 - age_output_acc: 0.3926 - weight_output_acc: 0.6329 - bag_output_acc: 0.5520 - footwear_output_acc: 0.5362 - pose_output_acc: 0.6192 - emotion_output_acc: 0.7092 - val_loss: 8.2828 - val_gender_output_loss: 0.6797 - val_image_quality_output_loss: 1.1513 - val_age_output_loss: 1.4410 - val_weight_output_loss: 0.9972 - val_bag_output_loss: 0.9390 - val_footwear_output_loss: 1.2189 - val_pose_output_loss: 0.9563 - val_emotion_output_loss: 0.8994 - val_gender_output_acc: 0.5681 - val_image_quality_output_acc: 0.4552 - val_age_output_acc: 0.3827 - val_weight_output_acc: 0.6347 - val_bag_output_acc: 0.5654 - val_foo

In [24]:
res = model.evaluate_generator(valid_gen,verbose=1)
dict(zip(model.metrics_names,res))

106/106 [==============================] - 12s 118ms/step


{'age_output_acc': 0.41037735849056606,
 'age_output_loss': 1.3774694789130733,
 'bag_output_acc': 0.6438679245283019,
 'bag_output_loss': 0.8219265426104924,
 'emotion_output_acc': 0.71875,
 'emotion_output_loss': 0.8755890718608532,
 'footwear_output_acc': 0.6014150943396226,
 'footwear_output_loss': 0.920513575931765,
 'gender_output_acc': 0.8309257075471698,
 'gender_output_loss': 0.3779877577469034,
 'image_quality_output_acc': 0.5728183962264151,
 'image_quality_output_loss': 0.9291597795936296,
 'loss': 6.822923178942698,
 'pose_output_acc': 0.8045400943396226,
 'pose_output_loss': 0.5581984247239131,
 'weight_output_acc': 0.6359080188679245,
 'weight_output_loss': 0.9620785460157214}